In [1]:
!pip install pandas

In [1]:
import sys
print(sys.executable)



c:\Users\rahul\AppData\Local\Programs\Python\Python312\python.exe


In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import gc
import logging

In [6]:



# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_table_and_insert(csv_file_path, table_name, host, user, password, database, batch_size=100):
    try:
        # Connect to MySQL
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        cursor = connection.cursor(buffered=True)

        # Table structure based on the provided columns
        create_table_query = """
        CREATE TABLE IF NOT EXISTS `chicago` (
            `ID` INT,
            `Case Number` VARCHAR(50),
            `Date` DATETIME,
            `Block` VARCHAR(255),
            `IUCR` VARCHAR(10),
            `Primary Type` VARCHAR(100),
            `Description` VARCHAR(255),
            `Location Description` VARCHAR(255),
            `Arrest` BOOLEAN,
            `Domestic` BOOLEAN,
            `Beat` INT,
            `District` INT,
            `FBI Code` VARCHAR(10),
            `X Coordinate` FLOAT,
            `Y Coordinate` FLOAT,
            `Year` INT,
            `Updated On` DATETIME,
            `Latitude` FLOAT,
            `Longitude` FLOAT,
            `Location` VARCHAR(255)
        );
        """
        cursor.execute(create_table_query)
        logging.info(f"Table `{table_name}` created successfully!")

        # Process CSV in chunks to avoid memory issues
        total_rows_inserted = 0
        for chunk in pd.read_csv(csv_file_path, chunksize=batch_size):
            placeholders = ', '.join(['%s'] * len(chunk.columns))
            insert_query = f"INSERT INTO `{table_name}` VALUES ({placeholders})"
            cursor.executemany(insert_query, chunk.values.tolist())
            connection.commit()
            total_rows_inserted += cursor.rowcount
            logging.info(f"Inserted {cursor.rowcount} rows.")

            # Clear memory after processing each chunk
            del chunk
            gc.collect()  # Run garbage collection to free up memory

        logging.info(f"Total rows inserted: {total_rows_inserted}")

    except Error as e:
        logging.error(f"Error: {e}")
    except Exception as ex:
        logging.error(f"Unexpected error: {ex}")
    finally:
        if 'connection' in locals() and connection.is_connected():
            cursor.close()
            connection.close()
            logging.info("MySQL connection closed.")

# Example Usage
create_table_and_insert(
    csv_file_path="D:/Chicago_crime-analyzer/crime_cleaned.csv",  # Path to your CSV file
    table_name="chicago",                                        # Table name
    host="localhost",                                            # MySQL host
    user="root",                                                 # MySQL username
    password="",                                    # MySQL password
    database="crime",                                            # MySQL database
    batch_size=1000                                               # Number of rows per batch
)


2024-12-26 16:48:52,004 - INFO - Table `chicago` created successfully!
2024-12-26 16:48:52,104 - INFO - Inserted 1000 rows.
2024-12-26 16:48:52,231 - INFO - Inserted 1000 rows.
2024-12-26 16:48:52,360 - INFO - Inserted 1000 rows.
2024-12-26 16:48:52,481 - INFO - Inserted 1000 rows.
2024-12-26 16:48:52,593 - INFO - Inserted 1000 rows.
2024-12-26 16:48:52,723 - INFO - Inserted 1000 rows.
2024-12-26 16:48:52,862 - INFO - Inserted 1000 rows.
2024-12-26 16:48:52,995 - INFO - Inserted 1000 rows.
2024-12-26 16:48:53,154 - INFO - Inserted 1000 rows.
2024-12-26 16:48:53,282 - INFO - Inserted 1000 rows.
2024-12-26 16:48:53,422 - INFO - Inserted 1000 rows.
2024-12-26 16:48:53,554 - INFO - Inserted 1000 rows.
2024-12-26 16:48:53,686 - INFO - Inserted 1000 rows.
2024-12-26 16:48:53,831 - INFO - Inserted 1000 rows.
2024-12-26 16:48:53,965 - INFO - Inserted 1000 rows.
2024-12-26 16:48:54,101 - INFO - Inserted 1000 rows.
2024-12-26 16:48:54,186 - INFO - Inserted 1000 rows.
2024-12-26 16:48:54,272 - IN